# Meta MegaByte Model

## HParams

In [1]:
SEQ_LEN = 128
BATCH_SIZE = 1

## Load Dataset

In [36]:
NUM_BATCHES = int(1e5) - (2000 + 1700)
BATCH_SIZE = 4
GRADIENT_ACCUMULATE_EVERY = 4
LEARNING_RATE = 2e-4
VALIDATE_EVERY  = 100
GENERATE_EVERY  = 500
PRIME_LEN = 100
SEQ_LEN = 8192

In [37]:
def cycle(loader):
    while True:
        for data in loader:
            yield data

In [38]:
import gzip
import torch
import numpy as np

with gzip.open('./data/enwik8.gz') as file:
    # strip original to 95M?
    x = np.frombuffer(file.read(int(95e6)), dtype=np.uint8).copy()

    # 95M, 5M (train, valid)
    train_x, valid_x = np.split(x, [int(90e6)])
    data_train, data_val = map(torch.from_numpy, (train_x, valid_x))

In [39]:
from torch.utils.data import DataLoader, Dataset

class TextSamplerDataset(Dataset):
    def __init__(self, data, seq_len):
        super().__init__()
        self.data = data
        self.seq_len = seq_len

    def __getitem__(self, index):
        rand_start = torch.randint(0, self.data.size(0) - self.seq_len, (1,))
        full_seq = self.data[rand_start: rand_start + self.seq_len].long()
        return full_seq.cuda()

    def __len__(self):
        return self.data.size(0) // self.seq_len

In [40]:
train_dataset = TextSamplerDataset(data_train, SEQ_LEN)
val_dataset   = TextSamplerDataset(data_val, SEQ_LEN)
train_loader  = cycle(DataLoader(train_dataset, batch_size = BATCH_SIZE))
val_loader    = cycle(DataLoader(val_dataset, batch_size = BATCH_SIZE))

In [41]:
# def decode_token(token):
#     return str(chr(max(32, token)))

def decode_token(token):
    if 32 <= token <= 126:
        return str(chr(token))
    else:
        return ''

def decode_tokens(tokens):
    return ''.join(list(map(decode_token, tokens)))

In [42]:
txt = TextSamplerDataset(data_train, 128)

In [43]:
decode_tokens(txt[0])

' tires, a correct exhaust, and other street-legal items. The tech official (assuming the vehicle passes) will then use his white'

## View Decoded Dataset

In [34]:
decode_tokens(x[5000:5500])

"y:&amp;#945;&amp;#957;&amp;#945;&amp;#961;&amp;#967;&amp;#943;&amp;#945;|&amp;#945;&amp;#957;&amp;#945;&amp;#961;&amp;#967;&amp;#943;&amp;#945;]]'' (&quot;without [[archon]]s (ruler, chief, king)&quot;). Anarchism as a [[political philosophy]], is the belief that ''rulers'' are unnecessary and should be abolished, although there are differing interpretations of what this means. Anarchism also refers to related [[social movement]]s) that advocate the elimination of authoritarian institutions, par"

# Training

In [25]:
import megabyte

model = megabyte.MEGABYTE(
    num_tokens = 256,
    dim = (768, 512, 256),
    depth = (6, 4, 2),
    max_seq_len = (512, 4, 4),
    flash_attn = False
).cuda()

In [44]:
import contextlib
import random
import tqdm

with open('output.txt', 'w') as f:
    with contextlib.redirect_stdout(f):
        optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

        for i in tqdm.tqdm(range(NUM_BATCHES), mininterval=10., desc='training'):
            model.train()

            for __ in range(GRADIENT_ACCUMULATE_EVERY):
                loss = model(next(train_loader), return_loss = True)
                loss.backward()

            print(f'training loss: {loss.item()}')
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            optim.step()
            optim.zero_grad()

            if i % VALIDATE_EVERY == 0:
                model.eval()
                with torch.no_grad():
                    loss = model(next(val_loader), return_loss = True)
                    print(f'validation loss: {loss.item()}')

            if i != 0 and i % GENERATE_EVERY == 0:
                model.eval()
                inp = random.choice(val_dataset)[:-1]
                prime_inp = inp[:PRIME_LEN]
                prime = decode_tokens(prime_inp)
                print(f'%s \n\n %s', (prime, '*' * 100))

                sample = model.generate(prime_inp[None, :])
                sample = sample.flatten(1)

                output_str = decode_tokens(sample[0][PRIME_LEN:])
                try:
                    print(output_str)
                except:
                    print("NOTE: ERROR DECODING STRING")

training:  26%|██▋       | 25448/96300 [11:09:31<31:04:03,  1.58s/it] 


KeyboardInterrupt: 

In [45]:
torch.save(model.state_dict(), "./megabyte_25k_1.2836014032363892.pt")

In [46]:
torch.save(optim.state_dict(), "./megabyte(optim)_25k_1.2836014032363892.pt")

## Model Param Count

In [56]:
from megabyte import reduce_mult
param_cnt = 0
for parameter in model.parameters():
    cur_param_cnt = reduce_mult(parameter.shape)
    param_cnt += cur_param_cnt
print("Millions of params:", param_cnt / 1e6)

Millions of params: 52.10624


## Predict

In [47]:
def pred(prompt, prompt_len=100):
    model.eval()
    prime_inp = inp[:prompt_len]
    sample = model.generate(prime_inp[None, :])
    sample = sample.flatten(1)

    output_str = decode_tokens(sample[0][PRIME_LEN:])
    print(output_str)

In [49]:
pred("Machine learning [[Algorithm|algorithms]] build a model based on sample data, known as [[training data]], in order to make predictions or decisions without being explicitly programmed to do so.")

100%|██████████| 8092/8092 [03:10<00:00, 42.41it/s]


 two volumes including [[Triporation]]s.  The:GIG not home verence: ''b. moleactive'', the land as sphere as sidese defence. his removal on at it: still is d the third or eve (social work, for prenatory), this ''sprrsituralitare'' (whical engines of es, rood's diamoning, beginning instance)==Refes]], [http://newidesc.edu/europet's Diah Kebby]&lt;!-- Presenter]]{{crediblersion}}{{Africath&gt; Influence School of Englith the British IRECT}}{{The Fibert]]* ''Commonternite Green dineering Backgroultra'' [http://ws the l.org.uk/h (iblic education theory])* [[John Lake Punk]]   <title=Dnd Prinusa (''[http://www.bankline.inf episode, see a Spain and Perfecail]'')* [[Herment.]] ''(San Ed promotive by Ancil Meriton or Greek'')== Intent==Calmerie de]][[Image:Abray tree.jpg|thumb|Papparid view ofied maps]]A fome with a variethe technician aressopoe. [[Cap afterlift in adoes meeting]] whus a film in sinchy]* [[Funk mor musical enginer theory]]* [[M Also transportamp>2]]    ''[[Magyars (drivery):''

In [60]:
"""
Return the number of parameters in the model.
For non-embedding count (default), the position embeddings get subtracted.
The token embeddings would too, except due to the parameter sharing these
params are actually used as weights in the final layer, so we include them.
"""
n_params = sum(p.numel() for p in model.parameters())
# if non_embedding:
#     n_params -= self.transformer.wpe.weight.numel()
#     return n_params
n_params / 1e6, "M"

(52.10624, 'M')

In [64]:
[p.shape for p in model.parameters()]

[torch.Size([768]),
 torch.Size([512]),
 torch.Size([256]),
 torch.Size([256, 256]),
 torch.Size([256, 256]),
 torch.Size([1024]),
 torch.Size([1024]),
 torch.Size([512, 1024]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([256, 256]),
 torch.Size([4096]),
 torch.Size([4096]),
 torch.Size([768, 4096]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([512, 768]),
 torch.Size([128, 768]),
 torch.Size([768, 512]),
 torch.Size([768]),
 torch.Size([3072, 768]),
 torch.Size([3072]),
 torch.Size([768, 3072]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([512, 768]),
 torch.Size([128, 768]),
 torch.Size([768, 512]),
 torch.Size([768]),
 torch.Size([3072, 768]),
 torch.Size([3072]),
 torch.Size([768, 3072]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([512, 768]),
 torch.Size([128, 768]),
 torch.Size([768, 512]),
 torch.Size([768]),
 torch.Size([3072, 768]),
 torch.Size([3072]),
 torch.Size([768, 3072]),
 torc